In [20]:
# Importing the libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
import pandas as pd
import numpy as np

In [13]:
# CNN Model
model = tf.keras.models.Sequential([
    Conv2D(16, (3,3), activation='relu', input_shape = (100, 100, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])
model.compile(optimizer = Adam(learning_rate = 0.01), loss = 'categorical_crossentropy', metrics = ['accuracy'], run_eagerly=True)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 49, 49, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 10, 10, 64)      

In [14]:
# Train data generator
train_dir = '/kaggle/input/state-farm-distracted-driver-detection/imgs/train'

train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size = (100, 100),
    class_mode = 'categorical',
    batch_size = 128
)

Found 22424 images belonging to 10 classes.


In [15]:
es = EarlyStopping(monitor='val_acc', patience = 2, min_delta=0.01)

In [16]:
model.fit_generator(train_generator, epochs=10, verbose=1, callbacks=[es])


/tmp/ipykernel_32/3003286466.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=10, verbose=1, callbacks=[es])


Epoch 1/10
176/176 [==============================] - 158s 897ms/step - loss: 1.4660 - accuracy: 0.5810
Epoch 2/10
176/176 [==============================] - 152s 862ms/step - loss: 0.2476 - accuracy: 0.9250
Epoch 3/10
176/176 [==============================] - 154s 874ms/step - loss: 0.1415 - accuracy: 0.9550
Epoch 4/10
176/176 [==============================] - 149s 847ms/step - loss: 0.1114 - accuracy: 0.9658
Epoch 5/10
176/176 [==============================] - 153s 866ms/step - loss: 0.1028 - accuracy: 0.9670
Epoch 6/10
176/176 [==============================] - 155s 881ms/step - loss: 0.0768 - accuracy: 0.9759
Epoch 7/10
176/176 [==============================] - 153s 870ms/step - loss: 0.0579 - accuracy: 0.9814
Epoch 8/10
176/176 [==============================] - 154s 871ms/step - loss: 0.0543 - accuracy: 0.9831
Epoch 9/10
176/176 [==============================] - 155s 877ms/step - loss: 0.0493 - accuracy: 0.9845
Epoch 10/10
176/176 [==============================] - 156s 884m

In [17]:
# creating the test directory in output folder
import os
os.mkdir('./test')
os.mkdir('./test/all_classes')

In [18]:
from shutil import copyfile

In [29]:
# copying from the input test folder to output test folder
d = pd.read_csv('/kaggle/input/state-farm-distracted-driver-detection/sample_submission.csv')
for row in d.values:
    file_name = row[0]
    copyfile( '/kaggle/input/state-farm-distracted-driver-detection/imgs/test/'+file_name , '/kaggle/working/test/all_classes/'+file_name )

In [30]:
# Test data generator
imageGen = ImageDataGenerator(rescale=1.0/255)
test_generator = imageGen.flow_from_directory(
    '/kaggle/working/test',
    class_mode=None,
    shuffle=False,
    target_size=(100,100),
)

Found 79726 images belonging to 1 classes.


In [31]:
# predicting the output for test data
test_generator.reset()
predictions = model.predict_generator(test_generator,verbose=1)

/tmp/ipykernel_32/2538877702.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(test_generator,verbose=1)


2492/2492 [==============================] - 218s 87ms/step


In [32]:
predictions.shape

(79726, 10)

In [33]:
# Converting the Prediction same as sample submission dataFrame
# Making the dictionary of The Predcition
d1 = pd.read_csv('/kaggle/input/state-farm-distracted-driver-detection/sample_submission.csv')
d ={
    'img':list(d1.values[:,0]),
}
for i in range(0,10):
    d['c' + str(i)] = list(predictions[:,i])

In [34]:
df = pd.DataFrame(d)
df

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_1.jpg,2.127454e-27,4.975863e-30,6.617141e-23,7.271837e-22,1.276044e-27,9.999999e-01,1.548674e-18,2.145076e-24,1.130061e-17,1.104193e-19
1,img_10.jpg,1.145451e-09,1.666201e-13,4.541736e-14,1.137352e-01,1.764223e-08,8.862647e-01,5.537083e-10,5.965526e-14,1.042204e-07,5.038670e-09
2,img_100.jpg,1.082155e-03,9.517600e-04,8.274587e-05,2.994509e-02,6.716945e-04,1.252460e-02,3.121231e-03,1.054190e-05,5.802068e-01,3.714033e-01
3,img_1000.jpg,6.373119e-05,2.698062e-01,1.862938e-06,9.381699e-09,1.076556e-06,1.627136e-08,6.955153e-01,1.654792e-07,3.456864e-02,4.299276e-05
4,img_100000.jpg,7.388915e-09,9.543239e-16,2.671545e-12,9.702330e-08,2.470559e-08,9.999970e-01,8.950802e-14,1.360060e-10,7.029488e-08,2.734672e-06
...,...,...,...,...,...,...,...,...,...,...,...
79721,img_99994.jpg,8.796642e-14,1.057379e-12,1.286250e-17,8.179086e-17,6.679139e-14,2.803035e-18,1.290185e-19,9.999999e-01,6.624996e-08,3.323026e-12
79722,img_99995.jpg,1.137694e-04,2.720131e-06,2.564093e-04,1.310796e-01,1.109061e-04,3.106649e-03,4.506606e-07,3.775062e-02,1.066134e-03,8.265127e-01
79723,img_99996.jpg,1.414758e-04,3.574874e-06,9.582720e-06,3.640197e-03,1.974567e-01,2.316184e-05,2.361212e-06,6.343993e-05,7.982491e-01,4.104258e-04
79724,img_99998.jpg,5.047594e-03,7.327678e-04,4.567176e-01,6.146956e-04,2.330826e-02,8.701496e-03,6.615076e-03,5.336231e-02,4.227180e-01,2.218220e-02


In [35]:
df.to_csv('./Prediction.csv',index=False)

In [37]:
# Getting Classes from Predictions
predicted_class_indices=np.argmax(predictions,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions_in_class = [labels[k] for k in predicted_class_indices]

In [45]:
filenames=test_generator.filenames
file=[]
for f in filenames:
    file.append(f.split('/')[1])
results=pd.DataFrame({"Filename":file ,
                      "Predictions":predictions_in_class})
results.to_csv('./Class_Prediction.csv',index=False)